# SC42x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드에 대해서 **한 줄**로 간단하게 요약해주세요. (세션 노트를 참고하여도 좋습니다.)<br/>
> **Tip : 아래 문제를 먼저 수행한 후 모델 학습 등 시간이 오래 걸리는 셀이 실행되는 동안 아래 내용을 작성하면 시간을 절약할 수 있습니다.**

**N421**
- **Stopwords(불용어)** : 분석에 큰 의미가 없는 단어
- **Stemming** : 단어에서 접사 등을 제거하고 단어의 의미를 담고 있는 단어의 핵심 부분을 남기는 작업
- **Lemmatization** : 단어의 뿌리 단어를 찾아가는 작업
- **Bag-of-Words** : 문서에서 문법이나 단어의 순서 등을 무시하고 단순히 단어들의 빈도만 고려하여 벡터화하는 방법
- **TF-IDF** : 특정 문서에만 등장하는 단어에 가중치를 두는 방법

**N422**
- **Word2Vec** : 단어를 벡터로 나타내는 방법
- **fastText** : Word2vec 기반으로 부분단어들을 임베딩하는 기법

**N423**
- **RNN** : 입력과 출력을 시퀀스 단위로 처리하는 모델
- **LSTM** : RNN에 기울기 정보 크기를 조절하기 위한 Gate를 추가한 모델
- **GRU** : LSTM을 간소화한 모델로 시간 측면에서 좀 더 빠름
- **Attention** : 디코더에서 출력 단어를 예측하는 시점마다, 인코더 입력 문장 중에서 해당 시점에서 예측해야할 단어와 연관이 있는 단어를 더 집중해서 보는 기법

# Part 2 : Fake/Real News Dataset

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다.<br/>
어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요.<br/>
이번 스프린트 챌린지에선 [Fake/Real News Dataset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)을 사용하여 배운 것들을 복습해보는 시간을 갖겠습니다.

**주의 : 모델의 성능을 최대한 끌어올리는 것이 아닌 모델 구동에 초점을 맞춰주세요.<br/>
모든 문제를 완료한 후에도 "시간이 남았다면" 정확도를 올리는 것에 도전하시는 것을 추천드립니다.**

In [28]:
# 코드 실행 전 seed를 지정하겠습니다.
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

## 2.0 데이터셋을 불러옵니다.

- 위 캐글 링크에서 데이터셋을 받아 업로드 합니다.<br/>
(직접 업로드하게 되면 시간이 꽤 걸리므로 **drive_mount** 나 **kaggle 연동**하시는 것을 추천드립니다.)

- 'label' 열을 만들어 Fake = 1, True = 0 로 레이블링해줍니다.
- 두 파일을 합쳐 하나의 데이터프레임에 저장해 준 후 데이터를 섞어줍니다.

In [4]:
# 파일 저장
from google.colab import files
files.upload()

Saving True.csv to True.csv


In [29]:
# 파일 불러오기
import pandas as pd

df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [30]:
# df_fake 정보 확인
df_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [31]:
# df_true 정보 확인
df_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [32]:
# label 생성
df_fake['label'] = 1
df_true['label'] = 0

# 두 데이터프레임 연결
df = pd.concat([df_fake, df_true], ignore_index = True)

# 데이터프레임 셔플
df = df.sample(frac = 1).reset_index(drop = True)

# 사용할 데이터프레임 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [33]:
# 데이터프레임 내용을 직접 확인
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0


## 2.1 TF-IDF 를 활용하여 특정 뉴스와 유사한 뉴스 검색하기

시간상 특별한 **전처리 없이** 아래 태스크를 수행하겠습니다.

### 2.1.1 TFidfVectorizer를 사용하여 문서-단어 행렬(Document-Term Matrix) 만들기

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 생성
tfidf_vect = TfidfVectorizer(stop_words = 'english', max_features = 100)

# dtm 생성
dtm_tfidf_news = tfidf_vect.fit_transform(df['text'])

dtm_tfidf_news = pd.DataFrame(dtm_tfidf_news.todense(), columns = tfidf_vect.get_feature_names_out())
dtm_tfidf_news

,000,2016,according,administration,america,american,americans,asked,called,campaign,...,washington,way,wednesday,week,white,women,work,world,year,years
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.188896,0.000000,0.000000,0.000000,0.000000,0.0,0.234041,0.000000,0.000000,0.0
1,0.0,0.000000,0.000000,0.183190,0.044892,0.000000,0.000000,0.000000,0.041490,0.041127,...,0.075784,0.042245,0.046241,0.000000,0.240405,0.0,0.000000,0.045513,0.035526,0.0
2,0.0,0.000000,0.000000,0.347947,0.000000,0.160179,0.000000,0.177877,0.000000,0.000000,...,0.000000,0.000000,0.175657,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,0.0,0.000000,0.152444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.178402,0.000000,0.074386,0.294941,...,0.000000,0.000000,0.000000,0.074934,0.071836,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
44894,0.0,0.097009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.084908,0.092938,0.000000,0.000000,0.0,0.000000,0.000000,0.071404,0.0
44895,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.700265,0.000000,0.000000,0.0
44896,0.0,0.000000,0.091836,0.000000,0.000000,0.000000,0.218133,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.101366,0.000000,0.175668,0.0,0.000000,0.000000,0.000000,0.0


### 2.1.2 KNN 알고리즘을 사용하여 유사한 문서 검색하기

- **42번 인덱스의 문서**와 가장 유사한 **5개 문서(42번 포함)의 인덱스**와 **해당 인덱스의 레이블**을 나타내주세요.
- NN 모델의 파라미터 중 `algorithm = 'kd_tree'` 로 설정합니다.

In [35]:
from sklearn.neighbors import NearestNeighbors

# NN 모델 생성
nn = NearestNeighbors(n_neighbors = 5, algorithm = 'kd_tree')
nn.fit(dtm_tfidf_news)

# 유사한 문서 검색
nn.kneighbors([dtm_tfidf_news.iloc[42]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


(array([[0.        , 0.        , 0.79745612, 0.83997222, 0.87030389]]),
 array([[33954,    42, 36664, 31854,  5235]]))

## 2.2 Keras Embedding을 사용하여 분류하기

### 2.2.0 데이터셋 split

- Train, Test 데이터셋으로 분리(Split)하여 주세요.

In [36]:
from sklearn.model_selection import train_test_split

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state = 42)

### 2.2.1 단어 벡터의 평균을 이용하여 분류해보기

N422에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길고 시간이 오래 걸리므로 시간상 epoch 수를 **10 이하**로 하는 것을 추천드립니다.<br/>
모델 구동이 목적이므로 임베딩 차원 수를 크지 않게(50이하)로 설정해주세요.<br/>
**권장사항 : `max_len` 은 텍스트 길이 평균보다 높게 설정해주세요.**<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [14]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [15]:
import gensim
import gensim.downloader as api

print(gensim.__version__)
wv = api.load('word2vec-google-news-300')

4.2.0
[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [37]:
import numpy as np

# max_len 길이 확인
print(f'텍스트 길이의 평균: {np.mean([len(sent) for sent in X_train], dtype=int)}')

텍스트 길이의 평균: 2465


In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# keras의 tokenizer에 텍스트를 학습
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(X_train)

# 텍스트 인코딩 진행
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded = tokenizer.texts_to_sequences(X_test)

# 단어 집합의 크기를 확인
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 :', vocab_size)

# 패딩 처리
max_len = 2500
X_train = pad_sequences(X_train_encoded, maxlen = max_len, padding='post')
X_test = pad_sequences(X_test_encoded, maxlen = max_len, padding='post')

단어 집합의 크기 : 125516


In [39]:
# word2vec의 임베딩 가중치 행렬 생성
embedding_matrix = np.zeros((vocab_size, 300))
print(np.shape(embedding_matrix))

# 입력 단어가 vocab에 있는 단어일 경우 임베딩 벡터를 반환하는 함수 작성
def get_vector(word):
    if word in wv:
        return wv[word]
    else:
        return None

# 함수를 적용하여 embedding_matrix 생성
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

(125516, 300)


In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

# 신경망 구성
model = Sequential()
model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_len, trainable = False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

# 모델 학습
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
model.fit(X_train, y_train, batch_size = 64, epochs = 3, validation_split = 0.2)

Epoch 1/3
449/449 [==============================] - 4s 7ms/step - loss: 0.6830 - acc: 0.5266 - val_loss: 0.6760 - val_acc: 0.5146
Epoch 2/3
449/449 [==============================] - 3s 7ms/step - loss: 0.6673 - acc: 0.5758 - val_loss: 0.6615 - val_acc: 0.6719
Epoch 3/3
449/449 [==============================] - 3s 6ms/step - loss: 0.6534 - acc: 0.7056 - val_loss: 0.6483 - val_acc: 0.7215


In [56]:
# 모델 평가 진행
eval = model.evaluate(X_test, y_test)

281/281 [==============================] - 1s 3ms/step - loss: 0.6474 - acc: 0.7229


### 2.2.2 LSTM을 사용하여 텍스트 분류 수행해보기

N423에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길어 시간이 매우 오래 걸리므로 <br/>
**층을 최소한으로 쌓고**, epoch 수를 **3 이하**로 하는 것을 추천드립니다.<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [61]:
from keras.layers import LSTM, Activation, Dropout, Input
from keras.models import Model
from keras.optimizers import RMSprop

# 모델 생성
def RNN():
    inputs = Input(name = 'inputs',shape = [max_len])
    layer = Embedding(vocab_size, 50, input_length = max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name = 'FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name = 'out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 2500)]            0         
                                                                 
 embedding_13 (Embedding)    (None, 2500, 50)          6275800   
                                                                 
 lstm_2 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [62]:
# 모델 학습
model.fit(X_train, y_train, batch_size = 64, epochs = 3, validation_split = 0.2)

Epoch 1/3
449/449 [==============================] - 44s 93ms/step - loss: 0.6924 - accuracy: 0.5220 - val_loss: 0.6942 - val_accuracy: 0.5146
Epoch 2/3
449/449 [==============================] - 42s 93ms/step - loss: 0.7006 - accuracy: 0.5234 - val_loss: 0.6931 - val_accuracy: 0.5146
Epoch 3/3
449/449 [==============================] - 39s 87ms/step - loss: 0.6997 - accuracy: 0.5243 - val_loss: 0.6929 - val_accuracy: 0.5146


In [63]:
# 모델 평가
eval = model.evaluate(X_test, y_test)

281/281 [==============================] - 9s 30ms/step - loss: 0.6914 - accuracy: 0.5245


### 2.2.3 위에서 실행한 내용에 대해 다시 알아봅시다.

#### a) 데이터셋을 학습할 때 사용하는 `pad_sequences`  메서드에 대해 설명해주세요.<br/>어떤 기능을 하나요? 모델을 학습할 때 왜 필요한가요?

*자연어 처리를 하다보면 각 문장의 길이가 서로 다를 수 있는데, 기계는 길이가 전부 동일한 문서들에 대해 하나의 행렬로 보고 한꺼번에 처리가 가능하다. 따라서, pad_sequences를 통해 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업을 함으로써, 병렬 연산이 가능하게 한다.*

#### b) 2.2.1과 2.2.2에서 사용한 각 모델의 evaluation 성능은 어떻게 나왔나요?<br/>각 모델의 장단점은 무엇이라고 생각하나요?

*2.2.1의 모델* 
1. embedding_matrix를 사용한 경우 정확도 : 0.7229
2. embedding_matrix를 사용하지 않은 경우 정확도 : 0.5246
3. 장점<br>
    - 구현이 간편
    - 간단한 문제에 대해서는 성능이 좋음
4. 단점
    - 문장에서의 단어 위치정보를 이용할 수 없음

*2.2.2의 모델* 
1. 정확도 : 0.5245
2. 장점
    - 각각의 메모리와 결과값이 컨트롤이 가능
3. 단점
    - 메모리가 덮어씌워질 가능성이 있음
    - 연산 속도가 느림


#### c) 종래의 RNN(Recurrent Neural Networks) 대신 LSTM(Long-Short Term Memory)을 사용하는 이유는 무엇인가요?<br/>(i.e. RNN에 비해 LSTM의 좋은 점을 설명해주세요.)

*기존의 RNN은 단기 메모리만을 가지고 학습을 진행해서 장기 의존성의 문제가 존재했다. 하지만 LSTM은 단기 메모리와 장기 메모리를 나눠 학습 후, 두 메모리를 병합해 이벤트 확률을 예측하기 때문에 과거의 정보를 훨씬 잘 반영한다는 장점이 있다.*

#### d) LSTM이나 RNN을 사용하는 예시를 **3개**이상 제시하고 해당되는 경우에 왜 LSTM이나 RNN을 사용하는 것 적절한지 간단하게 설명해주세요.

사용 예시 : 챗봇, 기계 번역, 음성 인식 등 <br>
사용 이유 : LSTM이나 RNN이 속하는 Sequence Model이란, 연속적인 입력으로부터 연속적인 출력을 생성하는 모델로서 순서가 있는 Sequence data에서 특징들을 추출하여 여러가지 문제를 해결하고 예측하기 때문에

#### e) 이외에 N424 에서 배운 자연어처리 모델과 관련된 키워드를 3개 이상 적어주세요. <br/> (해당 키워드에 대한 설명은 옵션입니다.)

*GPT 모델, BERT 모델, T5 모델*

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- 2.1 에서 TF-IDF(`TfidfVectorizer`)가 아닌 방법을 사용하여 유사도 검색을 수행해보세요.<br/>
TF-IDF와 해당 방법의 차이를 설명해주세요. 
- 2.2 에서 사용한 방법을 재사용하되 하이퍼 파라미터를 조정하거나 모델 구조를 변경하여 성능을 올려봅시다.<br/>**(주의 : GridSearch, RandomSearch 등의 방법을 사용하여도 좋으나 시간이 오래 걸리므로 범위를 잘 선택해야 합니다.)**

In [ ]:
# 이 곳에 답안을 작성하시길 바랍니다